# Pass CMP% On Motioning Plays

This notebook runs a quick a dirty test to see if the completion rate of passes thrown differs when there is motion on the play.

In [1]:
import pandas as pd
import data

## Test 1

This first test analyzes the CMP% difference arising from pre-snap motion.

First, let's create a `pd.DataFrame` for each targetted player play that indicates if there was motion on the play, if they were the motioning receiver, and if they caught the pass.

In [2]:
# load the player play data
player_play = pd.read_csv("../data/player_play.csv")

# make columns into bools
player_play["wasTargettedReceiver"] = player_play["wasTargettedReceiver"].fillna(0) == 1
player_play["hadPassReception"] = player_play["hadPassReception"].fillna(0) == 1
player_play["motionSinceLineset"] = player_play["motionSinceLineset"].fillna(0) == 1

# create a series that indicates whether there was motion on the play
wasMotion = (
    player_play.groupby(by=["gameId", "playId"])["motionSinceLineset"].sum() >= 1
)

# get only the important columns
targetted = player_play[player_play["wasTargettedReceiver"]][
    [
        "gameId",
        "playId",
        "nflId",
        "motionSinceLineset",
        "hadPassReception",
        "yardageGainedAfterTheCatch",
        "receivingYards",
    ]
].copy()


targetted["passYards"] = (
    targetted["receivingYards"] - targetted["yardageGainedAfterTheCatch"]
)


# create the `wasMotionOnPlay` column
def was_motion(row: pd.Series) -> bool:
    return wasMotion[row["gameId"], row["playId"]]


targetted["wasMotionOnPlay"] = targetted.apply(was_motion, axis=1)

targetted.head()

,gameId,playId,nflId,motionSinceLineset,hadPassReception,yardageGainedAfterTheCatch,receivingYards,passYards,wasMotionOnPlay
2,2022090800,56,42489,True,True,1,6,5,True
72,2022090800,122,47857,False,True,9,6,-3,False
90,2022090800,167,42489,False,True,6,12,6,False
140,2022090800,212,52494,False,True,2,8,6,True
162,2022090800,236,52536,True,True,11,26,15,True


Now let's group by whether the player motioned and whether there was motion on that play at all and look at some of the summary statistics.

In [3]:
targetted.groupby(by=["wasMotionOnPlay", "motionSinceLineset"])[
    [
        "hadPassReception",
        "passYards",
        "yardageGainedAfterTheCatch",
        "receivingYards",
    ]
].agg(["mean", "count"])

hadPassReception       passYards        \
                                               mean count      mean count   
wasMotionOnPlay motionSinceLineset                                          
False           False                      0.649466  4961  4.024793  4961   
True            False                      0.663592  2277  4.093983  2277   
                True                       0.786596  1134  2.505291  1134   

                                   yardageGainedAfterTheCatch        \
                                                         mean count   
wasMotionOnPlay motionSinceLineset                                    
False           False                                3.255997  4961   
True            False                                3.475187  2277   
                True                                 4.680776  1134   

                                   receivingYards        
                                             mean count  
wasMotionOnPlay motionSinceLineset                       
False           False                    7.280790  4961  
True            False                    7.569170  2277  
                True                     7.186067  1134

Findings:

* Motion on the play and then throwing to a receiver that motioned drastically increases the completion rate

    * This is probably because there are a lot of pitches to motioning receivers which is why the average passing yards to motioning receivers is low

* Motioning on the play and then throwing to a non-motioning receiver might increase the completion rate (currently a $p$-value of about $0.16$)

    * If this is significant it is probably because some corners get distracted by motion. Maybe we can explore more into this and see if there are occasional patterns backs exhibit that indicate they are too focused on the motioning receiver before the snap and not the receiver they need to be covering. I imagine if this is the case, and maybe small percentage of the time when a receiver is motioning a back shows signs of focusing on that receiver, the copmletion rate to the non-motioning receiver he's supposed to cover increases significantly above the $0.65$ average.

## Test 2

He we analyze how CMP% is affected when a non-motioning receivers surrounding defenders show some sort of response to the pre-snap motion.

Let's first load in all the data.

In [ ]:
# load players and play_play
players = pd.read_csv("../data/players.csv")
player_play = pd.read_csv("../data/player_play.csv")

# make columns into bools
player_play["wasTargettedReceiver"] = player_play["wasTargettedReceiver"].fillna(0) == 1
player_play["hadPassReception"] = player_play["hadPassReception"].fillna(0) == 1
player_play["motionSinceLineset"] = player_play["motionSinceLineset"].fillna(0) == 1

# create a series that indicates whether there was motion on the play
wasMotion = (
    player_play.groupby(by=["gameId", "playId"])["motionSinceLineset"].sum() >= 1
)

# get only the important columns
targetted = player_play[player_play["wasTargettedReceiver"]][
    [
        "gameId",
        "playId",
        "nflId",
        "motionSinceLineset",
        "hadPassReception",
        "yardageGainedAfterTheCatch",
        "receivingYards",
    ]
].copy()


targetted["passYards"] = (
    targetted["receivingYards"] - targetted["yardageGainedAfterTheCatch"]
)


# create the `wasMotionOnPlay` column
def was_motion(row: pd.Series) -> bool:
    return wasMotion[row["gameId"], row["playId"]]


targetted["wasMotionOnPlay"] = targetted.apply(was_motion, axis=1)

# create a column for the surrounded defenders response to the motion
#   - a quick a dirty approach to this is to average the movement of the 4 closest backs


def defender_response(row: pd.Series) -> float:
    if not row["wasMotionOnPlay"]:
        return 0
    elif row["motionSinceLineset"]:
        return 0
    else:
        # get the tracking for this game and this play
        gid = row["gameId"]
        tracking = data.load_tracking(gid)
        this_play = tracking["playId"] == row["playId"]
        selected_play = pd.merge(
            tracking[this_play], players[["nflId", "position"]], on="nflId"
        )

        # get the four closest defenders
        DEFENDER_POS = ["CB", "SS", "FS", "OLD", "ILB"]
        defenders = selected_play[selected_play["position"].isin(DEFENDER_POS)]


targetted["def_response"] = targetted.apply(defender_response)

targetted.head()

KeyError: 'wasMotionOnPlay'